In [24]:
import os
import numpy as np
import re
import math

from gurobipy import Model, GRB, GurobiError
import matplotlib.pyplot as plt
import numpy as np

#from graph_tool.generation import complete_graph
#from graph_tool.flow import boykov_kolmogorov_max_flow, min_st_cut

In [25]:
def get_opt_sol(problem):
    problem_short = problem.split('.')[0]
    if 'atsp' in problem:
        file = 'data/opt_atsp.txt'
    else:
        file = 'data/opt_tsp.txt'
    
    with open(file, 'r') as file:
        content = file.read()
    
    lines = content.split('\n')
    for line in lines:
        if line.split(':')[0].strip() == problem_short:
            return int(line.split(':')[1].strip())
    return None

In [61]:
DIR = 'data'
#n = 17
# 'att532.tsp'
problem = 'gr431.tsp' #'dsj1000.tsp' #'rbg323.atsp' # 'ftv64.atsp' #'br17.atsp'
file_name = os.path.join(DIR, problem)

if 'atsp' in problem:
    problem_type = 'atsp'
else:
    problem_type = 'tsp'

numeric_part = re.search(r'\d+', problem).group()

# Convert the extracted numeric part to an integer
n = int(numeric_part)
print(n)
if n == 64:
    n = 65
    
with open(file_name, 'r') as file:
    content = file.read()
    
print('Optimal solution:', get_opt_sol(problem))

431
Optimal solution: 171414


In [62]:
def calculate_distance(node1, node2):
    x1, y1 = node1
    x2, y2 = node2
    return math.ceil(math.sqrt((x2 - x1)**2 + (y2 - y1)**2))

def haversine_distance(coord1, coord2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of the Earth in kilometers (you can adjust this value)
    R = 6371.0

    # Calculate the distance
    distance = R * c

    return distance

In [63]:
data_lines = content.split('\n')

try:
    start_index = data_lines.index('EDGE_WEIGHT_SECTION')
except:
    start_index = data_lines.index('NODE_COORD_SECTION')
    
# Extract the data lines after 'EDGE_WEIGHT_SECTION'
edge_weight_lines = data_lines[start_index + 1:][:-2]

# Parse the distance matrix (lines are not rows)
if problem_type == 'atsp':
    distance_matrix = [list(map(int, line.split())) for line in edge_weight_lines if line.strip()]

    def flatten(l):
        return [item for sublist in l for item in sublist]

    distance_list = flatten(distance_matrix)
    print(len(distance_list))

    my_matrix = np.array(distance_list).reshape((n, n))
    my_matrix.shape
else:
    try:
        print('has_index')
        ys = [ float(line.split(' ')[3])  for line in edge_weight_lines if line.strip()]
        xs = [ float(line.split(' ')[2])  for line in edge_weight_lines if line.strip()]
    except:
        ys = [ float(line.split(' ')[2])  for line in edge_weight_lines if line.strip()]
        xs = [ float(line.split(' ')[1])  for line in edge_weight_lines if line.strip()]

has_index


In [64]:
for line in data_lines:
    if 'EDGE_WEIGHT_TYPE' in line:
        weight_type = line.split(':')[1].strip()
print(weight_type)

GEO


In [65]:
V = range(n)

if problem_type == 'atsp':
    cost = {
        (i, j): my_matrix[i,j]
        for i in V for j in V if i != j
    }
else:
    if weight_type == 'GEO':
        cost = {
            (i, j): haversine_distance((xs[i], ys[i]), (xs[j], ys[j]))
            for i in V for j in V if i != j
        }
    else:
        cost = {
            (i, j): math.sqrt((xs[i] - xs[j]) ** 2 + (ys[i] - ys[j]) ** 2)
            for i in V for j in V if i != j
        }

A = list(cost.keys())

In [66]:
def subtour_starting_at(vertex):
    tour = [vertex]
    current = next_after(vertex)

    while current != vertex:
        tour.append(current)
        current = next_after(current)

    return tour

def add_sec_for(subtour):
    #print('Added a violated SEC')
    global subtours_size
    
    n = len(set(subtour))
    #print(subtours_size)
    if n in subtours_size:
        subtours_size[n] = subtours_size[n] + 1
    else:
        subtours_size[n] = 1
    #print(set(subtour))
    m.cbLazy(sum(x[i, j] for i in subtour for j in set(V) - set(subtour)) >= 1)

def next_after(i):
    for j in V:
        if j == i:
            continue
        try:
            if m.cbGetSolution(x[i, j]) > 0.5:
                return j
        except GurobiError:
            if x[i, j].X > 0.9:
                return j
    
    assert False, f"Vertex {i} has no successor"

def callback(what, where):
    if where != GRB.Callback.MIPSOL:
        return

    remaining = set(V)

    while len(remaining) > 0:
        current = next(iter(remaining))
        subtour = subtour_starting_at(vertex=current)

        if len(subtour) == n:
            return
        
        add_sec_for(subtour)

        remaining -= set(subtour)

In [67]:
global subtours_size
subtours_size = {}

m = Model()
x = m.addVars(A, vtype=GRB.BINARY, name='x', obj=cost)
m.addConstrs((x.sum(i, '*') == 1 for i in V), name='out_degree')
m.addConstrs((x.sum('*', i) == 1 for i in V), 'in_degree');
m.setParam(GRB.Param.LazyConstraints, 1)
m.setParam('TimeLimit', 15*60)
m.optimize(callback)

Set parameter LazyConstraints to value 1
Set parameter TimeLimit to value 900
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 862 rows, 185330 columns and 370660 nonzeros
Model fingerprint: 0x18dbb6f5
Variable types: 0 continuous, 185330 integer (185330 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.24s
Presolved: 862 rows, 185330 columns, 370660 nonzeros
Variable types: 0 continuous, 185330 integer (185330 binary)

Root relaxation: objective 1.601365e+05, 1135 iterations, 0.08 seconds (0.13 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 160136.455    0  340          - 160136.455      -

In [69]:
import gurobipy as gp

tour = subtour_starting_at(0)
#tour

obj_val =  m.objVal
print('Objective value:', obj_val)

# Get the execution time
exec_time = m.Runtime
print("Execution Time:", exec_time)

lazy_constraints = sum(subtours_size.values())
print(lazy_constraints)
print(subtours_size)

Objective value: 172042.9886197543
Execution Time: 481.0675151348114
1916
{272: 1, 40: 6, 12: 18, 41: 5, 51: 4, 15: 9, 2: 823, 3: 147, 5: 65, 122: 1, 4: 180, 9: 27, 8: 21, 46: 4, 7: 27, 6: 31, 65: 6, 10: 18, 20: 14, 16: 8, 17: 11, 14: 7, 25: 16, 236: 2, 11: 20, 38: 5, 74: 4, 21: 10, 55: 3, 44: 3, 26: 8, 83: 4, 13: 20, 22: 9, 27: 12, 92: 4, 30: 9, 31: 10, 23: 6, 125: 1, 35: 6, 42: 5, 39: 4, 29: 7, 164: 1, 37: 5, 48: 7, 129: 1, 90: 4, 49: 7, 111: 2, 28: 8, 60: 6, 63: 4, 69: 3, 99: 3, 19: 5, 76: 3, 34: 6, 246: 2, 287: 1, 33: 7, 52: 2, 47: 7, 54: 2, 86: 2, 141: 5, 96: 2, 24: 6, 18: 8, 32: 5, 56: 3, 187: 1, 77: 3, 168: 2, 43: 4, 36: 3, 67: 3, 162: 1, 160: 2, 193: 1, 127: 3, 66: 2, 197: 2, 142: 1, 71: 2, 106: 3, 81: 4, 103: 4, 109: 2, 45: 2, 233: 2, 159: 2, 163: 3, 130: 3, 114: 3, 70: 4, 72: 3, 156: 1, 138: 2, 235: 1, 78: 3, 95: 1, 73: 3, 82: 1, 203: 1, 102: 1, 116: 1, 171: 1, 98: 2, 151: 3, 91: 1, 94: 1, 177: 1, 174: 1, 181: 1, 84: 2, 364: 3, 57: 1, 310: 1, 59: 1, 419: 2, 128: 2, 50: 1, 194

In [71]:
subtours_size[3]

147

In [86]:
def my_model():
    m = Model()
    x = m.addVars(A, vtype=GRB.BINARY, name='x', obj=cost)
    m.addConstrs((x.sum(i, '*') == 1 for i in V), name='out_degree')
    m.addConstrs((x.sum('*', i) == 1 for i in V), 'in_degree');
    m.optimize(callback)

In [87]:
import timeit

execution_times = timeit.repeat(stmt=my_model, setup="pass", repeat=5, number=1)

# Calculate mean and standard deviation
mean_time = sum(execution_times) / len(execution_times)
std_time = (sum((time - mean_time) ** 2 for time in execution_times) / len(execution_times)) ** 0.5

print(f"Mean Execution Time: {mean_time} seconds")
print(f"Standard Deviation: {std_time} seconds")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 646 rows, 104006 columns and 208012 nonzeros
Model fingerprint: 0x6de01d01
Variable types: 0 continuous, 104006 integer (104006 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 5884.0000000
Presolve time: 0.17s
Presolved: 646 rows, 104006 columns, 208012 nonzeros
Variable types: 0 continuous, 104006 integer (104006 binary)

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Obj        Time
       0          0     infinity      0.0000000e+00      0s
       1        855   2.5000000e+08   1.2350000e+02      0s
       2       1423   2.3800001e+08   2.4050000e+02      0s
       3       1713   2.1800003e+

# Fractional method

In [66]:
def callback(what, where):
    if where not in (GRB.Callback.MIPSOL, GRB.Callback.MIPNODE):
        return
    
    if where == GRB.Callback.MIPNODE and m.cbGet(GRB.Callback.MIPNODE_STATUS) != GRB.OPTIMAL:
        return
    
    set_capacity()

    source = G.vertex(0)
    added  = set()

    for i in range(1, n):
        if i in added:
            continue

        sink = G.vertex(i)
        res = boykov_kolmogorov_max_flow(g=G, source=source, target=sink, capacity=cap)

        # Create an edge property map quickly by
        # copying an existing one.
        flow = res.copy()

        # The value held by the property map is in
        # the .a member. Because capacity == flow + residuals
        # we must write:
        flow.a = cap.a - res.a

        maxflow = sum(flow[a] for a in sink.in_edges())

        if maxflow < 1 - 1e-6:
            print(f"Violated SEC. Flow = {maxflow:.3f} < 1")
            cut = min_st_cut(g=G, source=source, capacity=cap, residual=res)

            assert cut[source] == True
            assert cut[sink] == False

            subtour = [j for j in V if cut[G.vertex(j)] == False]

            assert len(subtour) < n

            add_sec_for(subtour)

            added = added.union(subtour)
            
def set_capacity():
    for e in G.edges():
        i, j = e.source(), e.target()

        try:
            xval = m.cbGetSolution(x[i,j])
        except GurobiError:
            xval = m.cbGetNodeRel(x[i,j])

        cap[e] = xval

In [67]:
G = complete_graph(N=n, self_loops=False, directed=True)
cap = G.new_edge_property(value_type='double')
G.edge_properties['cap'] = cap

m = Model()
x = m.addVars(A, vtype=GRB.BINARY, obj=cost, name='x')
m.addConstrs(x.sum(i, '*') == 1 for i in V)
m.addConstrs(x.sum('*', i) == 1 for i in V)
m.setParam(GRB.Param.LazyConstraints, 1)

NameError: name 'complete_graph' is not defined

In [ ]:
m.optimize(callback=callback)

In [ ]:
import site

# List of directories where packages are installed
print(site.getsitepackages())

In [114]:
!conda install -c conda-forge graph-tool

done
Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::bleach==3.2.1=py_0
  - defaults/osx-64::zope.interface==5.1.2=py38haf1e3a3_0
  - defaults/osx-64::anaconda==2020.11=py38_0
  - defaults/osx-64::nose==1.3.7=py38_1004
  - defaults/osx-64::zope.event==4.5.0=py38_0
  - defaults/noarch::nbformat==5.0.8=py_0
  - defaults/noarch::jsonschema==3.2.0=py_2
  - defaults/noarch::nltk==3.5=py_0
  - defaults/noarch::isort==5.6.4=py_0
  - defaults/osx-64::cython==0.29.21=py38hb1e8313_0
  - defaults/noarch::backports.functools_lru_cache==1.6.1=py_0
  - defaults/noarch::pygments==2.7.2=pyhd3eb1b0_0
  - defaults/noarch::qtconsole==4.7.7=py_0
  - defaults/osx-64::pylint==2.6.0=py38_0
  - defaults/osx-64::astropy==4.0.2=py38haf1e3a3_0
  - defaults/noarch::seaborn==0.11.0=py_0
  - defaults/noarch::conda-repo-cli==1.0.4=pyhd3eb1b0_0
  - defaults/osx-64::conda-build==3.20.5=py38_1
  